In [4]:
# Fairness-Aware, Voice-First Job Matching Prototype
# Context: Rural India | Example Job: Welding / Metal Work
# Constraints: Simple Python, no external libraries

# -----------------------------
# Helper Functions
# -----------------------------

def yes_no_input(prompt):
    """Standard Yes/No input handler"""
    while True:
        ans = input(prompt + " (Yes/No): ").strip().lower()
        if ans in ["yes", "no"]:
            return ans == "yes"
        print("Please say Yes or No.")

def choice_input(prompt, choices):
    """Choice-based input"""
    print(prompt)
    for i, c in enumerate(choices, 1):
        print(f"{i}. {c}")
    while True:
        try:
            idx = int(input("Enter number: "))
            if 1 <= idx <= len(choices):
                return choices[idx - 1]
        except:
            pass
        print("Invalid choice. Try again.")

def number_input(prompt):
    """Numeric input"""
    while True:
        try:
            return int(input(prompt + ": "))
        except:
            print("Please enter a number.")

# -----------------------------
# 1. INPUT COLLECTION
# -----------------------------

print("\n--- Phone-Based Worker Assessment ---\n")

# A. Structural Vulnerability
structural_flags = {
    "Scheduled Caste (SC)": yes_no_input("Are you from a Scheduled Caste"),
    "Scheduled Tribe (ST)": yes_no_input("Are you from a Scheduled Tribe"),
    "Below Poverty Line (BPL)": yes_no_input("Is your household Below Poverty Line"),
    "Woman-headed household": yes_no_input("Is your household woman-headed"),
    "Physically challenged member": yes_no_input("Is there a physically challenged household member"),
    "Small/marginal farmer": yes_no_input("Are you a small or marginal farmer"),
    "Land reform beneficiary": yes_no_input("Are you a land reform beneficiary"),
    "Forest Rights Act beneficiary": yes_no_input("Are you a Forest Rights Act beneficiary"),
    "PMAY-G beneficiary": yes_no_input("Are you a PMAY-G beneficiary")
}

# B. Employment & Economic Distress
days_since_work = number_input("Days since last paid work")
days_worked_30 = number_input("Number of days worked in last 30 days")
seasonal_unemployment = yes_no_input("Is your work seasonal")
outstanding_debt = yes_no_input("Do you have outstanding debt")

# C. Household & Shock Factors
elderly = yes_no_input("Do you have an elderly dependent (60+)")
children = yes_no_input("Do you have children under 6")
medical_emergency = yes_no_input("Any medical emergency in last 3 months")
natural_shock = yes_no_input("Any recent natural shock (flood, drought, crop loss)")

# D. Skillset
primary_background = choice_input(
    "What is your primary work background?",
    ["Agriculture", "Construction", "Mechanical / Repair", "Factory / Workshop", "General Labour", "None"]
)

repair_exp = choice_input(
    "Have you repaired or helped repair machines?",
    ["Often", "Sometimes", "Only helped", "Never"]
)

tool_exp = choice_input(
    "Have you used tools like welding torch, grinder, drill?",
    ["Often", "Sometimes", "Only helped", "Never"]
)

independent_work = choice_input(
    "Can you work independently?",
    ["Yes", "With guidance", "No"]
)

# -----------------------------
# 2. SCORING LOGIC
# -----------------------------

# A. Support Score (0–100)
support_score = 0
support_breakdown = []

# Structural Vulnerability (max 50)
structural_points = 0
for k, v in structural_flags.items():
    if v:
        structural_points += 6  # equal weight assumption
        support_breakdown.append(k)
structural_points = min(structural_points, 50)
support_score += structural_points

# Employment & Economic Distress (max 30)
employment_points = 0
if days_worked_30 == 0:
    employment_points += 15
elif days_worked_30 < 10:
    employment_points += 10

if days_since_work > 30:
    employment_points += 10
elif days_since_work > 15:
    employment_points += 6

if seasonal_unemployment:
    employment_points += 3
if outstanding_debt:
    employment_points += 2

employment_points = min(employment_points, 30)
support_score += employment_points

# Household & Shock Factors (max 20)
household_points = 0
for flag in [elderly, children, medical_emergency, natural_shock]:
    if flag:
        household_points += 5

household_points = min(household_points, 20)
support_score += household_points

support_score = min(support_score, 100)

# B. Skill–Job Fit Score (Welding / Metal Work)
job_fit_score = 0

# Background adjacency
background_weights = {
    "Mechanical / Repair": 30,
    "Construction": 22,
    "Factory / Workshop": 20,
    "General Labour": 12,
    "Agriculture": 8,
    "None": 0
}
job_fit_score += background_weights[primary_background]

# Experience frequency
exp_map = {"Often": 20, "Sometimes": 12, "Only helped": 6, "Never": 0}
job_fit_score += exp_map[repair_exp]
job_fit_score += exp_map[tool_exp]

# Independence
ind_map = {"Yes": 10, "With guidance": 6, "No": 0}
job_fit_score += ind_map[independent_work]

job_fit_score = min(job_fit_score, 100)

# C. Skill Readiness Score (general employability)
readiness_score = 0

if primary_background != "None":
    readiness_score += 30

if independent_work == "Yes":
    readiness_score += 25
elif independent_work == "With guidance":
    readiness_score += 15

if tool_exp in ["Often", "Sometimes"]:
    readiness_score += 20
elif tool_exp == "Only helped":
    readiness_score += 10

# Worked at least once in last 6 months
if days_since_work <= 180:
    readiness_score += 25

readiness_score = min(readiness_score, 100)

# D. Overall Urgency Score
overall_urgency = (0.75 * support_score) + (0.25 * readiness_score)

# Urgency Band
if overall_urgency >= 85:
    band = "Extreme"
elif overall_urgency >= 70:
    band = "Very High"
elif overall_urgency >= 50:
    band = "High"
else:
    band = "Moderate"

# -----------------------------
# 3. OUTPUT REPORT
# -----------------------------

print("\n--- Worker Score Report ---\n")

print(f"Support Score: {support_score:.1f}/100")
print("Support Factors:")
if support_breakdown:
    for s in support_breakdown:
        print(f" - {s}")
else:
    print(" - No major structural vulnerabilities reported")

if days_worked_30 == 0:
    print(" - No work in the last 30 days")
if days_since_work > 30:
    print(" - Long gap since last paid work")

print(f"\nSkill–Job Fit Score (Welding / Metal Work): {job_fit_score:.1f}/100")
print("Explanation:")
print(f" - Background: {primary_background}")
print(f" - Machine repair experience: {repair_exp}")
print(f" - Tool familiarity: {tool_exp}")
print(f" - Independence level: {independent_work}")

print(f"\nSkill Readiness Score: {readiness_score:.1f}/100")

print(f"\nOverall Urgency Score: {overall_urgency:.1f}/100")
print(f"Urgency Band: {band}")

print("\nPlain-language explanation:")
if support_score >= 70:
    print("You are marked urgent because you face economic or social hardship and have limited recent work.")
elif readiness_score < 40:
    print("You may need some training or support before starting work.")
else:
    print("You are moderately urgent and may be matched with suitable opportunities.")

print("\n--- End of Report ---")


--- Phone-Based Worker Assessment ---



Are you from a Scheduled Caste (Yes/No):  yes
Are you from a Scheduled Tribe (Yes/No):  no
Is your household Below Poverty Line (Yes/No):  yes
Is your household woman-headed (Yes/No):  no
Is there a physically challenged household member (Yes/No):  yes
Are you a small or marginal farmer (Yes/No):  no
Are you a land reform beneficiary (Yes/No):  no
Are you a Forest Rights Act beneficiary (Yes/No):  yes
Are you a PMAY-G beneficiary (Yes/No):  yes
Days since last paid work:  23
Number of days worked in last 30 days:  12
Is your work seasonal (Yes/No):  yes
Do you have outstanding debt (Yes/No):  no
Do you have an elderly dependent (60+) (Yes/No):  yes
Do you have children under 6 (Yes/No):  yes
Any medical emergency in last 3 months (Yes/No):  no
Any recent natural shock (flood, drought, crop loss) (Yes/No):  yes


What is your primary work background?
1. Agriculture
2. Construction
3. Mechanical / Repair
4. Factory / Workshop
5. General Labour
6. None


Enter number:  3


Have you repaired or helped repair machines?
1. Often
2. Sometimes
3. Only helped
4. Never


...

Have you used tools like welding torch, grinder, drill?
1. Often
2. Sometimes
3. Only helped
4. Never


Enter number:  1


Can you work independently?
1. Yes
2. With guidance
3. No


Enter number:  1



--- Worker Score Report ---

Support Score: 54.0/100
Support Factors:
 - Scheduled Caste (SC)
 - Below Poverty Line (BPL)
 - Physically challenged member
 - Forest Rights Act beneficiary
 - PMAY-G beneficiary

Skill–Job Fit Score (Welding / Metal Work): 72.0/100
Explanation:
 - Background: Mechanical / Repair
 - Machine repair experience: Sometimes
 - Tool familiarity: Often
 - Independence level: Yes

Skill Readiness Score: 100.0/100

Overall Urgency Score: 65.5/100
Urgency Band: High

Plain-language explanation:
You are moderately urgent and may be matched with suitable opportunities.

--- End of Report ---
